## Ejercicio unión de bases de datos

## Contexto:
---

Los fabricantes de grandes máquinas analizan rutinariamente los materiales que utilizan. Este conjunto de datos contiene información sobre varios tubos utilizados en máquinas. Puedes imaginar que las grandes máquinas requieren diferentes tipos de tubos; los coches tienen tubos, los aviones tienen tubos y las excavadoras también tienen tubos. Hablemos un poco sobre cómo se ve un tubo en este conjunto de datos.

La parte más pequeña de un ensamblaje de tubos es un componente. Los componentes pueden ser cosas como abrazaderas, juntas o algún otro tipo de accesorios. En general, los ensamblajes de tubos consisten en uno o más componentes y también pueden ser diferentes combinaciones de componentes. En este ejercicio, analizarás los datos y responderás algunas preguntas sobre los pesos. Sin embargo, antes de poder analizar los datos, debes organizarlos en un formato manejable. Necesitarás usar lo que has aprendido sobre la combinación y agrupación de datos.

Tienes estos dos dataframes para responder las preguntas. Echa un vistazo y familiarízate con ellos:

- **component_counts**: contiene la cantidad de componentes en cada ensamblaje de tubos.
- **component_weights**: contiene los pesos de los componentes individuales.

### Preguntas a contestar 

1. Cuales son los 5 tubos más pesados? cuales son sus pesos?
2. Cuantos *tube_assembly_id* tienen más de cinco componentes? R: 120
3. Cuantos *component_id* son usados en más de 50 *tube_assemblies*? R:69
4. Cual es el peso promedio de los 5 componenentes más pesados? R: 95.4894

### Referencias
- https://github.com/timmyshen/Cat_Tube
- https://www.kaggle.com/code/crawford/python-groupby-and-merge-exercises/notebook


## Ejercicio
---
En este ejercicio, nos adentraremos en el análisis de datos relacionados con la fabricación de grandes máquinas, centrando nuestra atención en los tubos que las componen. El **objetivo** de este análisis es responder preguntas específicas sobre los pesos de estos tubos y sus componentes. Para ello, trabajaremos con dos dataframes clave: `component_counts`, que registra la cantidad de componentes por ensamblaje de tubos, y `component_weights`, que contiene los pesos individuales de los componentes.

Nuestro enfoque se centrará en combinar y agrupar estos datos para obtener insights valiosos, como identificar los tubos más pesados y analizar la distribución de componentes en los ensamblajes.

El primer paso será importar el ambiente y las dos distintas bases de datos. 

In [31]:
# Set del ambiente e importación del dataframe

## Importar paquetes
import pandas as pd

## Cargar base de datos
### Número de componentes
component_counts = pd.read_csv("../exercises/data/component_count.csv")

### Peso componentes
component_weights = pd.read_csv("../exercises/data/component_weights.csv")

### Inspección de los datos
Antes de realizar el análisis, es necesario realizar una inspección inicial de los datos. Esto nos permitirá entender mejor la estructura de los dataframes, identificar cualquier valor nulo o anómalo, y verificar la consistencia entre las columnas clave. Durante esta inspección,analizaremos las dimensiones.


In [32]:
# Información general de la tabla de conteo de componentes
## Información general 
component_counts.info()
print()

## Inspección general 
component_counts.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39459 entries, 0 to 39458
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   tube_assembly_id  39459 non-null  object 
 1   component_id      39459 non-null  object 
 2   component_count   39459 non-null  float64
dtypes: float64(1), object(2)
memory usage: 924.9+ KB



,tube_assembly_id,component_id,component_count
5178,TA-05666,C-1625,2.0
909,TA-00994,C-1622,2.0
15776,TA-17521,C-1312,2.0
36780,TA-12604,C-1638,1.0
33152,TA-19959,C-1628,2.0


In [33]:
# Información general de la tabla de peso de los componentes
## Información general 
component_weights.info()
print()

## Inspección general 
component_weights.sample(5)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2047 entries, 0 to 2046
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   component_id  2047 non-null   object 
 1   weight        2047 non-null   float64
dtypes: float64(1), object(1)
memory usage: 32.1+ KB



,component_id,weight
1529,C-0220,0.970
537,C-1248,0.018
1581,C-0480,0.000
204,C-0238,1.121
1813,C-1884,0.716


In [34]:
## Inspección valores nulos (Confirmación)
### Para número de componentes
print("Valores ausentes tabla número de componentes: ", component_counts.isna().sum().sum())

### Para peso componentes
print("Valores ausentes tabla peso de componentes: ", component_weights.isna().sum().sum())

Valores ausentes tabla número de componentes:  0
Valores ausentes tabla peso de componentes:  0


In [35]:
## Inspección valores duplicados

### Para número de componentes
print(component_counts["component_id"].value_counts())
print("Valores únivocos en tabla de número de componentes: ", component_counts["component_id"].nunique())
print("Valores duplicados en tabla de número de componentes: ", component_counts["component_id"].duplicated().sum())
print()

### Para peso de componentes 
print(component_weights["component_id"].value_counts())
print("Valores únivocos en tabla de peso de componentes: ", component_weights["component_id"].nunique())
print("Valores duplicados en tabla de peso de componentes: ", component_weights["component_id"].duplicated().sum())


component_id
C-1621    2114
C-1628    2103
C-1629    1965
C-1622    1933
C-1624    1585
          ... 
C-1049       1
C-0981       1
C-0967       1
C-0934       1
C-0663       1
Name: count, Length: 2048, dtype: int64
Valores únivocos en tabla de número de componentes:  2048
Valores duplicados en tabla de número de componentes:  37411

component_id
C-2033    1
C-0005    1
C-0006    1
C-1435    1
C-1546    1
         ..
C-1229    1
C-1198    1
C-1197    1
C-1196    1
C-1195    1
Name: count, Length: 2047, dtype: int64
Valores únivocos en tabla de peso de componentes:  2047
Valores duplicados en tabla de peso de componentes:  0


La inspección de la tabla `component_counts` muestra que cuenta con 39459 observaciones sin valores nulos. Dentro de estas observaciones, se identificaron 2048 `component_id` distintos. Aunque no existen duplicados explícitos en términos de filas idénticas, algunos `component_id` se repiten con frecuencia. Un ejemplo es el `component_id` "C-1621", que aparece 2114 veces.

Por otro lado, la tabla `component_weights` consta de 2047 observaciones, sin valores nulos y sin duplicados.

Estas tablas están relacionadas mediante la columna `component_id`. Aunque hay una gran cantidad de `component_id`, podemos inferir que no hay variaciones de peso para un mismo componente, lo cual simplificará el análisis posterior.

### Manipulación de los datos
En esta etapa, nos centramos en la manipulación de los datos mediante la unión de las dos tablas: `component_counts` y `component_weights`,  ambas relacionadas por la columna component_id. Como resultado de esta unión tendrémos una nueva tabla `component_df`, facilitando el análisis posterior sobre los pesos de los tubos y sus componentes.

In [36]:
## Unión de las bases de datos
### Creación de la base de datos nueva
component_df = pd.merge(left=component_counts, right= component_weights, how = "left", on="component_id")

Después de realizar la unión, es útil inspeccionar las primeras filas del dataframe resultante para confirmar que la operación se haya realizado correctamente.

In [37]:
## Inspección del df compuesto
### Primeras filas
print(component_df.head())
print()

## Dimensión
print(component_df.shape)

  tube_assembly_id component_id  component_count  weight
0         TA-00001       C-1622              2.0   0.036
1         TA-00002       C-1312              2.0   0.009
2         TA-00003       C-1312              2.0   0.009
3         TA-00004       C-1312              2.0   0.009
4         TA-00005       C-1624              1.0   0.035

(39459, 4)


Como podemos observar, tenemos una tabla similar a la tabla del número de componentes pero con la adición del peso de cada componente. A partir de esto se puede proseguir con la exploración de datos para tratar de responder las preguntas. Es importante tener en cuenta que esta nueva base tiene valores duplicados, esto ocurre porque en la tabla `component_counts`, un mismo `component_id` puede aparecer varias veces debido a que un componente puede estar presente en múltiples ensamblajes de tubos.

Estos "duplicados" son en realidad filas necesarias para el análisis, ya que representan la presencia del mismo componente en diferentes ensamblajes de tubos, dependerá del análisis que necesitemos para ver la manera en que trabajaremos con ellos. 

### Exploración de datos
Con los datos ya combinados, pasamos a la fase de análisis de datos. En esta etapa, nos enfocaremos en responder las preguntas planteadas al inicio. Utilizaremos técnicas de agrupación, filtrado y ordenamiento para extraer insights valiosos sobre los tubos y sus componentes.

In [38]:
# 1. Conocer los 5 tubos más pesados y con sus respectivos pesos
## Ordenar la tabla de mayor a menor
component_df_sorted = component_df.sort_values(by = "weight", ascending = False)

## Eliminar los duplicados
heavy_id = component_df_sorted.drop_duplicates(subset="component_id")

print(heavy_id.head(5))

      tube_assembly_id component_id  component_count  weight
38201         TA-18226       C-1290              1.0   13.77
29561         TA-14536       C-1050              1.0   12.90
26561         TA-10360       C-0794              1.0   12.03
31038         TA-16670       C-0404              1.0   10.56
24580         TA-07626       C-0646              1.0   10.19


**Nota**
La primera opción que intentamos fue eliminar los duplicados y ordenar los valores. Sin embargo, posteriormente nos piden el valor promedio de los 5 componenentes más pesados, por lo que no se puede usar la misma base de datos ya filtrando los valores únicos. Por ello primero decidimos ordenar las filas de mayor a menor y eliminar los duplicados para el primer ejercicio, eso no compromete responder la pregunta posterior. 

In [39]:
# 2. Cuantos *tube_assembly_id* tienen más de cinco componentes? R: 120
## Agrupar el número de componentes por id de ensamblaje
component_num_by_assembly = component_df.groupby(by= "tube_assembly_id")["component_count"].sum()

## Filtrar los ensamblajes que tengan mas de 5 componentes 
popular_assembly = component_num_by_assembly[component_num_by_assembly > 5]

## Contar e impirmir el número de ensamblajes
print(popular_assembly.count()) 


120


In [40]:
# 3. Cuantos *component_id* son usados en más de 50 *tube_assemblies*? R:69
## Agrupar los ensamblajes por id de componente y contabilizarlos
assembly_by_component = component_df_sorted.groupby(by="component_id")["tube_assembly_id"].count()

## Filtrar los componentes con mas de 50 ensamblajes
popular_component = assembly_by_component[assembly_by_component > 50].sort_values(ascending = False)

## Determinar longiitud de la lista
print(len(popular_component))

69


In [41]:
# 4. Cual es el peso promedio de los 5 componenentes más pesados? R: 95.4894
# Obtener los primeros 5 valores de heavy_id
top_heavy_id = heavy_id.iloc[:5]

## Extaer los componentes más pesados
heavy_component_lst = top_heavy_id["component_id"].tolist()

## Seleccionar los componentes más pesados del df
heavy_component_df = component_df_sorted[component_df_sorted["component_id"].isin(heavy_component_lst)]

## Determinar promedio de los componentes más pesados
print(heavy_component_df["weight"].mean())



11.457142857142857


En este análisis, identificamos los cinco tubos más pesados: C-1290, C-1050, C-0794, C-0404 y C-0646. Además, determinamos que 120 ensamblajes contienen más de cinco componentes y que 69 componentes son utilizados en más de 50 ensamblajes. Finalmente, calculamos el peso promedio de los componentes más pesados, obteniendo un valor de 11.58. Este resultado difiere del valor esperado de 95.48, lo que sugiere un posible malentendido en la interpretación de la pregunta.

Desde mi perspectiva, el peso promedio de los componentes más pesados no debería superar los valores observados en la primera pregunta, que oscilan entre 10.19 y 13.77. Por lo tanto, sería recomendable revisar esta cuestión con el profesor.

Este análisis nos ha permitido comprender mejor la distribución y características de los tubos, lo que a su vez facilitará la optimización en su selección y uso en diversas aplicaciones industriales.